In [10]:
from requests import post
from os import getenv
from datetime import datetime
import json
import pydantic

# NOTE: don't for get to set "apikey" env, or the default below.
resp = post(
    "https://forecast-v2.metoceanapi.com/point/time",
    headers={"x-api-key": getenv("metservice_key", "MYAPIKEY")},
    json={
        "points": [{
            "lon": 174.7842,
            "lat": -37.7935
        }],
        "variables": [
            "cloud.cover",
        ],
        "time": {
            "from": "{:%Y-%m-%dT00:00:00Z}".format(datetime.now()),
            "interval": "3h",
            "repeat": 2
        }
    }
)

if resp.status_code != 200:
    raise ValueError("{}: {}", resp.status_code, resp.text)

print(json.dumps(resp.json(), indent=1))]

{
 "dimensions": {
  "point": {
   "type": "point",
   "units": "unknown",
   "data": [
    {
     "lon": 174.7842,
     "lat": -37.7935
    }
   ]
  },
  "time": {
   "type": "time",
   "units": "unknown",
   "data": [
    "2024-02-07T00:00:00Z",
    "2024-02-07T03:00:00Z",
    "2024-02-07T06:00:00Z"
   ]
  }
 },
 "noDataReasons": {
  "ERROR_INTERNAL": 2,
  "FILL": 3,
  "GAP": 1,
  "GOOD": 0,
  "INVALID_HIGH": 5,
  "INVALID_LOW": 4
 },
 "variables": {
  "cloud.cover": {
   "standardName": "cloud_area_fraction",
   "units": "percent",
   "dimensions": [
    "time",
    "point"
   ],
   "data": [
    1.4381456,
    0,
    0
   ],
   "noData": [
    0,
    0,
    0
   ]
  }
 }
}


In [4]:
from requests import post, get
from os import getenv
from datetime import datetime
import json

# NOTE: don't for get to set "apikey" env, or the default below.
resp = get(
    "https://forecast-v2.metoceanapi.com/point/time",
    headers={"x-api-key": getenv("metservice_key", "MYAPIKEY")},
    json={
        "points": [{
            "lon": 174.7842,
            "lat": -37.7935
        }],
        "variables": [
            "cloud.cover",
        ],
        "time": {
            "from": "{:%Y-%m-%dT00:00:00Z}".format(datetime.now()),
            "interval": "3h",
            "repeat": 2
        }
    }
)
print(f"resp type: {type(resp)}")
if resp.status_code != 200:
    raise ValueError("{}: {}", resp.status_code, resp.text)

print(json.dumps(resp.json(), indent=1))

resp type: <class 'requests.models.Response'>


ValueError: ('{}: {}', 400, '\n<html><head>\n<meta http-equiv="content-type" content="text/html;charset=utf-8">\n<title>400 Bad Request</title>\n</head>\n<body text=#000000 bgcolor=#ffffff>\n<h1>Error: Bad Request</h1>\n<h2>Your client has issued a malformed or illegal request.</h2>\n<h2></h2>\n</body></html>\n')

In [17]:
from typing import List, Dict, Union
from pydantic import BaseModel, Field


class PointData(BaseModel):
    lon: float
    lat: float


class DimensionData(BaseModel):
    type: str
    units: str
    # Union to support both 'point' and 'time' dimensions
    data: Union[List[PointData], List[str]]


class VariableDimension(BaseModel):
    standardName: str
    units: str
    dimensions: List[str]
    # Adjusted to support float, int, and None types
    data: List[Union[float, int, None]]
    noData: List[int]


class PointsResponseModel(BaseModel):
    dimensions: Dict[str, DimensionData]
    noDataReasons: Dict[str, int]
    variables: Dict[str, VariableDimension]


parsed_response = PointsResponseModel.model_validate_json(json.dumps(resp.json()))

In [18]:
print(parsed_response.dimensions)

{'point': DimensionData(type='point', units='unknown', data=[PointData(lon=174.7842, lat=-37.7935)]), 'time': DimensionData(type='time', units='unknown', data=['2024-02-07T00:00:00Z', '2024-02-07T03:00:00Z', '2024-02-07T06:00:00Z'])}


In [23]:
from typing import List, Optional

class PointTimeRequest(BaseModel):
    lat: float = Field(..., ge=-90, le=90)
    lon: float = Field(..., ge=-180, le=360)
    variables: List[str]
    from_date: Optional[str] = Field(None, alias='from')
    interval: Optional[str]
    repeat: Optional[int]
    models: Optional[List[str]]
    baseModels: Optional[List[str]]
    outputFormat: Optional[str]
    cycleLock: Optional[str]
    explain: Optional[bool] = False
    extrapolate: Optional[bool] = True
    kind: Optional[str]
    joinModels: Optional[str]

In [24]:
json_schema = PointTimeRequest.model_json_schema()

In [25]:
print(json_schema)

{'properties': {'lat': {'maximum': 90.0, 'minimum': -90.0, 'title': 'Lat', 'type': 'number'}, 'lon': {'maximum': 360.0, 'minimum': -180.0, 'title': 'Lon', 'type': 'number'}, 'variables': {'items': {'type': 'string'}, 'title': 'Variables', 'type': 'array'}, 'from': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': None, 'title': 'From'}, 'interval': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'title': 'Interval'}, 'repeat': {'anyOf': [{'type': 'integer'}, {'type': 'null'}], 'title': 'Repeat'}, 'models': {'anyOf': [{'items': {'type': 'string'}, 'type': 'array'}, {'type': 'null'}], 'title': 'Models'}, 'baseModels': {'anyOf': [{'items': {'type': 'string'}, 'type': 'array'}, {'type': 'null'}], 'title': 'Basemodels'}, 'outputFormat': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'title': 'Outputformat'}, 'cycleLock': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'title': 'Cyclelock'}, 'explain': {'anyOf': [{'type': 'boolean'}, {'type': 'null'}], 'default': False, 'ti